# Topic modeling

## Comparación entre Propuesta de Constitución y Constitución Vigente

### Librerías

In [1]:
import pandas as pd
import numpy as np

In [2]:
# for text preprocessing
import re
import spacy

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [3]:
# import LDA from sklearn
from sklearn.decomposition import LatentDirichletAllocation

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
esp_stop = stopwords.words('spanish')
esp_stop = esp_stop + ['ser','tener','deber']

## Funciones

In [6]:
def delete_stopwords(text):
    """
    Función para eliminar stopwords del texto
    """
    doc = nlp(text)
    return ' '.join([word.text for word in doc if word.text not in esp_stop])

In [22]:
def cleaner(df, text):
    """
    Función para dejar las palabras en minúscula, eliminar números, puntuación, espacios y saltos de línea.
    """
    df[text] = df[text].str.lower()
    df[text] = df[text].apply(lambda x: re.sub("[0-9]", '', str(x)))
    df[text] = df[text].str.translate(str.maketrans(' ', ' ', string.punctuation))
    df[text] = df[text].str.replace('\n',' ')
    df[text] = df[text].str.replace('nº','')
    df[text] = df[text].str.replace('n°','')
    df[text] = df[text].str.replace('n º','')
    df[text] = df[text].str.replace('n °','')
    df[text] = df[text].str.replace('º','')
    df[text] = df[text].str.replace('°','')
    df[text] = df[text].str.strip()

In [23]:
def wordcloud(data):
    """
    Función para graficar un wordcloud
    """
    wc = WordCloud(background_color='white',width=800, height=400, max_words=200, colormap = 'plasma').generate_from_frequencies(data)
    plt.figure(figsize=(15, 10))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.show()

### Importando textos ya digitalizados

In [24]:
df_propuesta = pd.read_csv('data/processed/constitucion_propuesta.csv')
df_propuesta.rename(columns = {'articulo':'nro_articulo'}, inplace = True)
df_propuesta['constitucion'] = 'Propuesta'
df_propuesta = df_propuesta[df_propuesta['capitulo']!='Convención Constitucional']

df_vigente = pd.read_csv('data/processed/constitucion_actual.csv')
df_vigente.drop(columns = 'id', inplace = True)
df_vigente['constitucion'] = 'Vigente'

df = pd.concat([df_vigente,df_propuesta])

df = df[['constitucion','capitulo', 'titulo_capitulo', 'nro_articulo', 'texto']]

In [25]:
df.sample(3)

,constitucion,capitulo,titulo_capitulo,nro_articulo,texto
126,Vigente,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 19,Nadie puede ser obligado a pertenecer a una as...
387,Propuesta,Capítulo III,Naturaleza y Medioambiente,Artículo 127,2. El Estado debe adoptar una administración e...
1420,Propuesta,Capítulo X,Órganos Autónomos Constitucionales,Artículo 307,4. La ley determinará las demás atribuciones...


### Desarrollo

Basado en: https://github.com/sethns/Latent-Dirichlet-Allocation-LDA-/blob/main/Topic%20Modeling%20_%20Extracting%20Topics_%20Using%20Sklearn.ipynb

Limpiando texto

In [31]:
cleaner(df, 'texto')

In [32]:
df['texto_clean'] = df['texto'].apply(lambda x: delete_stopwords(x))

In [33]:
df.sample()

,constitucion,capitulo,titulo_capitulo,nro_articulo,texto,texto_clean
1722,Propuesta,Disposiciones Transitorias,Disposiciones Transitorias,Artículo 383,cuadragésima segunda,cuadragésima segunda


In [49]:
df[df['constitucion']=='Propuesta']['texto_clean'][150]

'podrá participar diseño construcción rehabilitación conservación innovación vivienda considerará particularmente diseño políticas vivienda personas bajos ingresos económicos pertenecientes grupos especial protección'

Definiendo nlp

In [34]:
nlp = spacy.load('es_core_news_sm')

In [35]:
cv_vectorizer = CountVectorizer(ngram_range = (1,1))
cv_arr = cv_vectorizer.fit_transform(df['texto_clean'])

In [36]:
cv_arr

<2619x6117 sparse matrix of type '<class 'numpy.int64'>'
	with 42277 stored elements in Compressed Sparse Row format>

In [38]:
# Creating vocabulary array which will represent all the corpus 
vocab_cv = cv_vectorizer.get_feature_names()

# get the vocb list
vocab_cv[:10]

['abandono',
 'abastecer',
 'abastecimiento',
 'abierta',
 'abiertamente',
 'abierto',
 'abiertos',
 'abogada',
 'abogadas',
 'abogado']

### Implementando LDA

In [39]:
# Implementation of LDA:
    
# Create object for the LDA class 
# Inside this class LDA: define the components:
lda_model = LatentDirichletAllocation(n_components = 6, max_iter = 20, random_state = 20)

# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = lda_model.fit_transform(cv_arr)

# .components_ gives us our topic distribution 
topic_words = lda_model.components_



In [40]:
topic_words

array([[2.1648418 , 1.16666631, 1.166915  , ..., 2.16602604, 0.16833759,
        0.16666699],
       [0.16666857, 0.16666677, 1.15735854, ..., 0.16666782, 0.16666888,
        0.1666673 ],
       [0.17179423, 0.16666678, 0.16666885, ..., 1.16664394, 0.16666909,
        0.16689617],
       [0.16762829, 0.16666671, 0.16666754, ..., 0.16732766, 2.16600864,
        0.16666695],
       [0.16666764, 0.16666672, 0.17803652, ..., 0.16666729, 0.16970118,
        2.16612739],
       [1.16239947, 0.16666672, 1.16435356, ..., 0.16666725, 1.16261462,
        0.16697519]])

In [42]:
#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 5

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_cv)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

Topic 1 ['ley' 'arti' 'culo' 'constitucional']
Topic 2 ['an' 'regional' 'os' 'podra']
Topic 3 ['corte' 'artículo' 'caso' 'inciso']
Topic 4 ['presidente' 'congreso' 'diputados' 'si']
Topic 5 ['derecho' 'derechos' 'persona' 'toda']
Topic 6 ['ley' 'constitucio' 'pu' 'blica']


In [44]:
vocab_cv

['abandono',
 'abastecer',
 'abastecimiento',
 'abierta',
 'abiertamente',
 'abierto',
 'abiertos',
 'abogada',
 'abogadas',
 'abogado',
 'abogados',
 'abordar',
 'aborde',
 'abril',
 'abrir',
 'abrirse',
 'absoluta',
 'absoluto',
 'absolutoria',
 'absuelta',
 'abuso',
 'abusos',
 'aca',
 'acade',
 'academia',
 'academias',
 'académica',
 'académicas',
 'académicos',
 'acatar',
 'acceder',
 'accesibilidad',
 'accesible',
 'acceso',
 'accidentes',
 'accio',
 'acciones',
 'acción',
 'aceptable',
 'aceptada',
 'aceptando',
 'aceptar',
 'aceptarse',
 'acepte',
 'acerca',
 'aclaración',
 'acoge',
 'acoger',
 'acogerse',
 'acogida',
 'acogiera',
 'acogimiento',
 'acoja',
 'acompan',
 'acompañamiento',
 'aconcagua',
 'acordada',
 'acordadas',
 'acordado',
 'acordados',
 'acordar',
 'acordaren',
 'acorde',
 'acoso',
 'acreditación',
 'acreditada',
 'acredite',
 'acrediten',
 'acta',
 'actas',
 'actitudinales',
 'activa',
 'actividad',
 'actividades',
 'activo',
 'activos',
 'acto',
 'actor',
 